In this notebook, we compute and aggregate statistics for the enrichment of conditions in cases versus controls, as well as in each cluster versus all the other clusters.

We begin by defining a generic function to compute enrichment given "cases" and "controls" AnnDatas.

In [ ]:
# imports
import numpy as np
import pandas as pd
import scanpy as sc
from glob import glob
from tqdm import tqdm
from scipy.stats import hypergeom

# generic function to compoute enrichment
def enrichment(cases, controls, save_path):
    
    # pull out conditions to iterate through
    conditions = cases.var["<concept ID>"].tolist()
    assert conditions == controls.var["<concept ID>"].tolist(), "Columns must match between cases and controls."
    n_conditions = len(conditions)
    
    # convert to column-based matrices for faster operations
    print("converting to CSC format...")
    cases_mat = cases.X.tocsc()
    controls_mat = controls.X.tocsc()
    
    # create dictionary to look up condition names
    condition_dict = cases.var.set_index("<concept ID>")["<concept name>"].to_dict()
    
    # iterate over conditions
    print("iterating over conditions...")
    results = []
    for i, c in enumerate(tqdm(conditions)):
        
        # create contingency table
        cases_with = cases_mat[:, i].sum()
        controls_with = controls_mat[:, i].sum()
        obs = np.array([
            [cases_with, controls_with],
            [cases_mat.shape[0] - cases_with, controls_mat.shape[0] - controls_with]
        ])
        
        # zero-count correction if needed
        if (obs == 0).any():
            obs += 1

        # compute odds ratio and bounds
        odds = (obs[0, 0] * obs[1, 1]) / (obs[0, 1] * obs[1, 0])
        width = 1.96 * np.sqrt((1 / obs[0, 0]) + (1 / obs[0, 1]) + (1 / obs[1, 0]) + (1 / obs[1, 1]))
        lower = np.exp(np.log(odds) - width)
        upper = np.exp(np.log(odds) + width)
        
        # set up hypergeometric distribution parameters
        # M: size of population
        # n: number of successes in population
        # N: size of selection
        # x: number of successes in selection
        M = cases.shape[0] + controls.shape[0]
        n = obs[0, 0] + obs[0, 1]
        if odds >= 1:
            N = cases.shape[0]
            x = obs[0, 0]
        else:
            N = controls.shape[0]
            x = obs[0, 1]
            
        # compute p-value and apply Bonferroni correction
        p = hypergeom(M=M, n=n, N=N).sf(x - 1)
        p_adj = p * n_conditions
        
        # record result
        results.append({
            "snomed": c,
            "name": condition_dict[c],
            "odds": odds,
            "lower": lower,
            "upper": upper,
            "p": p,
            "p-adj": p_adj,
            "sig": p_adj < 0.05,
            "cases_with": int(cases_with),
            "controls_with": int(controls_with),
            "cases_without": int(obs[1, 0]),
            "controls_without": int(obs[1, 1])
        })
        
    # convert to DataFrame and save
    print("saving results...")
    results = pd.DataFrame.from_records(results).sort_values(["p-adj", "odds"], ascending=[True, False]).reset_index(drop=True)
    results.to_csv(save_path, index=False)
    return results

We start by comparing all conditions across cases versus controls.

In [ ]:
# load full AnnData, remove endometriosis concepts
endo_concepts = pd.read_csv("<concept filepath>")["<concept ID>"].tolist()
adata = sc.read_h5ad("<AnnData filepath for all patients, all conditions>")
adata = adata[:, ~adata.var["<concept ID>"].isin(endo_concepts)]

# iterate over match replicates
for i in range(1, 31):
    
    # print progress
    print(f"\n===== replicate {i} of 30 =====\n")
    
    # subset to replicate and filter conditions
    print("subsetting AnnData and filtering conditions...")
    subad = adata[adata.obs["replicate"].isin((0, i))]
    sc.pp.filter_genes(subad, min_counts=1)
    
    # split cases and controls and run enrichment analysis
    cases = subad[subad.obs["endo"] == 1]
    controls = subad[subad.obs["endo"] == 0]
    _ = enrichment(cases, controls, f"<directory for full case versus control analysis>/replicate-{i:02}.csv")

We then compare pre-endometriosis conditions.

In [ ]:
# load AnnData
adata = sc.read_h5ad("<AnnData filepath for all patients, pre-endo conditions>")

# iterate over match replicates
for i in range(1, 31):
    
    # print progress
    print(f"\n===== replicate {i} of 30 =====\n")
    
    # subset to replicate and filter conditions
    print("subsetting AnnData and filtering conditions...")
    subad = adata[adata.obs["replicate"].isin((0, i))]
    sc.pp.filter_genes(subad, min_counts=1)
    
    # split cases and controls and run enrichment analysis
    cases = subad[subad.obs["endo"] == 1]
    controls = subad[subad.obs["endo"] == 0]
    _ = enrichment(cases, controls, f"<directory for pre-endo case versus control analysis>/replicate-{i:02}.csv")

Next, we compute aggregate statistics across the replicates for the case versus control analysis.

In [ ]:
def agg_results(directory, save_path):
    
    # load all results into a single DataFrame
    all_results = None
    for i in range(1, 31):
        df = pd.read_csv(f"{directory}/replicate-{i:02}.csv")
        df["replicate"] = [i] * df.shape[0]
        if all_results is None:
            all_results = df
        else:
            all_results = pd.concat([all_results, df], ignore_index=True)
    
    # get all conditions
    all_conditions = sorted(all_results["snomed"].unique().tolist())

    # get aggregate metrics for each conditions
    records = []
    for c in tqdm(all_conditions):
        df = all_results[all_results["snomed"] == c]
        agg_p_adj = df["p-adj"].mean() * 2
        records.append({
            "snomed": c,
            "name": df["name"].unique().item(),
            "avg-odds": df["odds"].mean(),
            "agg-p-adj": agg_p_adj,
            "agg-sig": agg_p_adj < 0.05,
            "sig-frac": df["sig"].mean(),
            "avg-cases-with": df["cases_with"].mean(),
            "avg-controls-with": df["controls_with"].mean(),
            "avg-cases-without": df["cases_without"].mean(),
            "avg-controls-without": df["controls_without"].mean()
        })
        
    # convert to DataFrame, save and return
    agg_results = pd.DataFrame.from_records(records).sort_values(["agg-p-adj", "avg-odds"], ascending=[True, False]).reset_index(drop=True)
    agg_results.to_csv(save_path, index=False)
    return agg_results

# run for case versus control analyses
agg_results("<directory for full case versus control analysis>", "<full case versus control results filepath>")
agg_results("<directory for pre-endo case versus control analysis>", "<pre-endo case versus control results filepath>")

For the clusters, we use the same functions but select different "case" and "control" groups based on the cluster under consideration.

In [ ]:
# analysis for clusters using all conditions
full_results = {}
adata = sc.read_h5ad("<AnnData filepath for endo patients, all conditions>")
adata = adata[:, ~adata.var["<concept ID>"].isin(endo_concepts)]
clusters = sorted(adata.obs["leiden"].unique().tolist())
for cluster in clusters:
    print(f"\n===== cluster {cluster} of {len(clusters)} =====\n")
    cluster_str = f"{int(cluster):02}"
    cases = adata[adata.obs["leiden"] == cluster]
    controls = adata[adata.obs["leiden"] != cluster]
    full_results[cluster_str] = enrichment(cases, controls, f"<directory for cluster analysis, all conditions>/cluster-{cluster_str}.csv")

# analysis for clusters using pre-endometriosis conditions
pre_results = {}
adata = sc.read_h5ad("<AnnData filepath for endo patients, pre-endo conditions>")
adata = adata[:, ~adata.var["<concept ID>"].isin(endo_concepts)]
clusters = sorted(adata.obs["leiden"].unique().tolist())
for cluster in clusters:
    print(f"\n===== cluster {cluster} of {len(clusters)} =====\n")
    cluster_str = f"{int(cluster):02}"
    cases = adata[adata.obs["leiden"] == cluster]
    controls = adata[adata.obs["leiden"] != cluster]
    pre_results[cluster_str] = enrichment(cases, controls, f"<directory for cluster analysis, pre-endo conditions>/cluster-{cluster_str}.csv")

For each cluster, we want to keep only the conditions that were exclusively significantly enriched in that cluster.

In [ ]:
# take a list of paths and make a version of each result with only exclusively enriched conditions
def make_unique(paths):
    
    # load in significant conditions for each path
    print("loading significant enrichments...")
    enrichments = {}
    for p in tqdm(paths):
        enrichments[p] = pd.read_csv(p)
        enrichments[p] = enrichments[p][enrichments[p]["sig"] == True]
        
    # keep only unique conditions
    print("finding unique conditions...")
    for p in tqdm(paths):

        # get initial set of conditions
        this_conditions = set(enrichments[p]["snomed"])

        # get set of all conditions from other clusters
        other_paths = paths.copy()
        other_paths.remove(p)
        other_conditions = set()
        for i in other_paths:
            other_conditions |= set(enrichments[i]["snomed"])

        # set difference to get conditions to keep
        keep_conditions = this_conditions - other_conditions

        # only keep those conditions
        enrichments[p] = enrichments[p][enrichments[p]["snomed"].isin(keep_conditions)]
        enrichments[p] = enrichments[p].sort_values(["p", "odds"], ascending=[True, False]).reset_index(drop=True)
        
    # write results to disk
    print("saving results...")
    for p in tqdm(paths):
        enrichments[p].to_csv(p[:-14] + "unique-" + p[-14:], index=False)

# run for cluster analyses
make_unique(sorted(glob("<directory for cluster analysis, all conditions>/cluster-*")))
make_unique(sorted(glob("<directory for cluster analysis, pre-endo conditions>/cluster-*")))